In [5]:
import torch
import model

## load checkpoint

In [6]:
classes = 5

net = model.ResnetBlock(block=model.BasicBlock, block_stride=[1, 2, 2, 2, 2, 2, 2, 2],
                            in_channel=1, out_channel=32, classes=classes)
net.load_state_dict(torch.load('./trained_model_5/100.pt'))
net.eval()

ResnetBlock(
  (head): Sequential(
    (conv1): Conv1d(1, 32, kernel_size=(33,), stride=(1,), padding=(16,), bias=False)
    (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU()
  )
  (layers): Sequential(
    (basicblock_0): BasicBlock(
      (conv1): Conv1d(32, 32, kernel_size=(13,), stride=(1,), padding=(6,), bias=False)
      (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU()
      (conv2): Conv1d(32, 32, kernel_size=(13,), stride=(1,), padding=(6,), bias=False)
      (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU()
      (maxpool): MaxPool1d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
    )
    (basicblock_1): BasicBlock(
      (conv1): Conv1d(32, 32, kernel_size=(13,), stride=(1,), padding=(6,), bias=False)
      (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_sta

## convert to onnx

In [13]:
example_inputs = torch.randn(1, 1, 3750)

onnx_export_path = 'trained_model_5.onnx'

torch.onnx.export(
    net.cpu(),
    example_inputs.cpu(),
    onnx_export_path,
    export_params=True,
    # dynamo=True,
    # input_names=["input"],
    # output_names=["reg_onset_output", "reg_offset_output", "frame_output"],
    # dynamic_axes = {
    #     "input"             : {0: "batch_size", 1: "audio_len"},
    #     "reg_onset_output"  : {0: "batch_size", 1: "time_steps"},
    #     "reg_offset_output" : {0: "batch_size", 1: "time_steps"},
    #     "frame_output"      : {0: "batch_size", 1: "time_steps"},
    # },
    opset_version=17,
    do_constant_folding=True,
)

c:\Github\ECGSleepNet\model.py:103: TracerWarning: Using len to get tensor shape might cause the trace to be incorrect. Recommended usage would be tensor.shape[0]. Passing a tensor of different shape might lead to errors or silently give incorrect results.
  self.current_batch = len(x)
c:\Users\murgu\.conda\envs\cuda11\lib\site-packages\torch\onnx\symbolic_opset9.py:4476: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(


============= Diagnostic Run torch.onnx.export version 2.0.0+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [14]:
import onnx
import onnxruntime as ort
from onnxruntime.quantization import quantize_dynamic, QuantType

# validate the export
onnx_model = onnx.load(onnx_export_path)
onnx.checker.check_model(onnx_model)

In [16]:
# validate the export
onnx_model = onnx.load(onnx_export_path)
onnx.checker.check_model(onnx_model)

# %%
# ceating an onnxruntime session
PROVIDER = 'CPUExecutionProvider' 

onnx_session = ort.InferenceSession(onnx_export_path, providers=[PROVIDER]) # CPU

x_onnx = example_inputs.cpu().numpy()
ort_inputs = {"x": x_onnx}
ort_outs = onnx_session.run(None, ort_inputs)


In [17]:
ort_outs

[array([[ 4.726025  , -1.0704551 , -1.4170607 , -0.83633757, -0.83888346]],
       dtype=float32)]